In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv


In [2]:
calendar = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv', parse_dates=['date'])

In [3]:
# Sort calendar by date 
calendar = calendar.sort_values(by='date')

In [4]:
def add_event_feats(event):
    _event_occ = calendar.loc[(calendar.event_name_1 == event) | (calendar.event_name_2 == event), 'date'].values
    _diff = (calendar.date.values[:,None] - _event_occ[None,:]) / np.timedelta64(1,'D')
    _lags = []
    for i in range(len(calendar)):
        _day_row = _diff[i,:]
        if _day_row.max() < 0:
            _lags.append(-1)
        else:
            _lags.append(_day_row[np.where(_day_row>=0)[0]].min())
    return np.array(_lags).astype(np.int16)

In [5]:
events = set()
for event in calendar.event_name_1.unique():
    if event is not np.nan:
        events.add(event)
for event in calendar.event_name_2.unique():
    if event is not np.nan:
        events.add(event)

In [6]:
len(events)

30

In [7]:
for event in events:
    calendar[f'{event}_lag'] = add_event_feats(event)

In [8]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,LentStart_lag,Thanksgiving_lag,Purim End_lag,VeteransDay_lag,Ramadan starts_lag,Mother's day_lag,NBAFinalsStart_lag,Easter_lag,NBAFinalsEnd_lag,LaborDay_lag
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,...,126,202,83,217,8,38,13,80,365,282
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,...,127,203,84,218,9,39,14,81,366,283
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,...,128,204,85,219,10,40,15,82,367,284
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,...,129,205,86,220,11,41,16,83,368,285


In [9]:
calendar.columns

Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'PresidentsDay_lag', 'Halloween_lag',
       'NewYear_lag', 'Christmas_lag', 'Cinco De Mayo_lag',
       'ValentinesDay_lag', 'EidAlAdha_lag', 'ColumbusDay_lag',
       'Chanukah End_lag', 'OrthodoxChristmas_lag', 'SuperBowl_lag',
       'MemorialDay_lag', 'Pesach End_lag', 'Father's day_lag',
       'StPatricksDay_lag', 'Eid al-Fitr_lag', 'LentWeek2_lag',
       'OrthodoxEaster_lag', 'MartinLutherKingDay_lag', 'IndependenceDay_lag',
       'LentStart_lag', 'Thanksgiving_lag', 'Purim End_lag', 'VeteransDay_lag',
       'Ramadan starts_lag', 'Mother's day_lag', 'NBAFinalsStart_lag',
       'Easter_lag', 'NBAFinalsEnd_lag', 'LaborDay_lag'],
      dtype='object')

In [10]:
calendar.to_csv('calendar_events.csv')

In [11]:
! head calendar_events.csv

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,PresidentsDay_lag,Halloween_lag,NewYear_lag,Christmas_lag,Cinco De Mayo_lag,ValentinesDay_lag,EidAlAdha_lag,ColumbusDay_lag,Chanukah End_lag,OrthodoxChristmas_lag,SuperBowl_lag,MemorialDay_lag,Pesach End_lag,Father's day_lag,StPatricksDay_lag,Eid al-Fitr_lag,LentWeek2_lag,OrthodoxEaster_lag,MartinLutherKingDay_lag,IndependenceDay_lag,LentStart_lag,Thanksgiving_lag,Purim End_lag,VeteransDay_lag,Ramadan starts_lag,Mother's day_lag,NBAFinalsStart_lag,Easter_lag,NBAFinalsEnd_lag,LaborDay_lag
0,2011-01-29,11101,Saturday,1,1,2011,d_1,,,,,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,,,,,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2011-01-31,11101,Monday,3,1,2011,d_3,,,,,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-

In [12]:
import pickle

In [13]:
with open('events.pkl', 'wb') as f:
    pickle.dump(events, f)